# Library

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna

from datetime import datetime
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Data Loading

In [12]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


# Feature Engineering

- Add new feature

In [13]:
def create_features(df):
    # キルとデスの比率
    df['killDeathRatio'] = df['blueKills'] / (df['blueDeaths'] + 1)  # ゼロ除算を防ぐために+1

    # アシストの割合
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)

    # ヘラルドの討伐数
    df['blueHerald'] = df['blueEliteMonsters'] - df['blueDragons']
    # blueHerald列の負の値を0で置き換え
    df['blueHerald'] = df['blueHerald'].clip(lower=0)

    # エリートモンスターとドラゴンの比率
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    # エリートモンスターとヘラルドの比率
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)

    # ゴールドと経験値の対数変換
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])

    # 初期キルの影響
    df['firstBloodKillRatio'] = df['blueFirstBlood'] * df['blueKills']
    
    # キル、デス、アシストの合計
    df['totalCombatPoints'] = df['blueKills'] + df['blueDeaths'] + df['blueAssists']

    # キル・デス・アシスト比率
    total_combat_actions = df['blueKills'] + df['blueDeaths'] + df['blueAssists']
    df['killRatio'] = df['blueKills'] / total_combat_actions
    df['deathRatio'] = df['blueDeaths'] / total_combat_actions
    df['assistRatio'] = df['blueAssists'] / total_combat_actions

    # チームの総合力
    df['teamStrength'] = df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']

    return df

# model

- data

In [14]:
# データの準備
X = train.drop(['blueWins', 'gameId'], axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

test_drop_gameId = test.drop('gameId', axis=1)

- optuna

In [15]:
# Optunaでのハイパーパラメータチューニング
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    verbose_eval = 0
    verbose = lgb.log_evaluation(verbose_eval)

    # 早期停止の設定
    early_stopping = lgb.early_stopping(stopping_rounds=50, verbose=True)
    model = lgb.train(param, lgb_train, num_boost_round=5000, valid_sets=[lgb_val], callbacks=[early_stopping, verbose])
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータでモデルを再学習
best_params = study.best_params
best_params['objective'] = 'binary'
best_params['metric'] = 'binary_error'
best_params['verbosity'] = -1

[I 2024-01-22 12:46:07,641] A new study created in memory with name: no-name-9d232555-242e-47a8-b5af-2d97053ba674


Training until validation scores don't improve for 50 rounds


[I 2024-01-22 12:46:08,264] Trial 0 finished with value: 0.8025 and parameters: {'lambda_l1': 0.001563880422307558, 'lambda_l2': 0.0003053748063124683, 'num_leaves': 6, 'feature_fraction': 0.7052287532824972, 'bagging_fraction': 0.8536071514044961, 'bagging_freq': 4, 'min_child_samples': 55}. Best is trial 0 with value: 0.8025.


Early stopping, best iteration is:
[188]	valid_0's binary_error: 0.1975
Training until validation scores don't improve for 50 rounds


[I 2024-01-22 12:46:08,596] Trial 1 finished with value: 0.808125 and parameters: {'lambda_l1': 4.9814494320025316e-05, 'lambda_l2': 0.02271150685992115, 'num_leaves': 150, 'feature_fraction': 0.6308929916189836, 'bagging_fraction': 0.9691809583904215, 'bagging_freq': 5, 'min_child_samples': 49}. Best is trial 1 with value: 0.808125.
[I 2024-01-22 12:46:08,728] Trial 2 finished with value: 0.809375 and parameters: {'lambda_l1': 0.0003465631157802406, 'lambda_l2': 0.004268732075661603, 'num_leaves': 101, 'feature_fraction': 0.8952339752215954, 'bagging_fraction': 0.6709701782690868, 'bagging_freq': 5, 'min_child_samples': 69}. Best is trial 2 with value: 0.809375.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[63]	valid_0's binary_error: 0.18625


[I 2024-01-22 12:46:28,583] Trial 98 finished with value: 0.81125 and parameters: {'lambda_l1': 4.2050365769302764e-05, 'lambda_l2': 1.9088347241858725e-05, 'num_leaves': 147, 'feature_fraction': 0.9193301018251541, 'bagging_fraction': 0.6482419247217409, 'bagging_freq': 2, 'min_child_samples': 79}. Best is trial 63 with value: 0.8175.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[92]	valid_0's binary_error: 0.18875


[I 2024-01-22 12:46:28,867] Trial 99 finished with value: 0.818125 and parameters: {'lambda_l1': 9.129256965834418e-05, 'lambda_l2': 3.185072147986896e-05, 'num_leaves': 165, 'feature_fraction': 0.9613164587931166, 'bagging_fraction': 0.6317993707158505, 'bagging_freq': 2, 'min_child_samples': 84}. Best is trial 99 with value: 0.818125.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[116]	valid_0's binary_error: 0.181875


- add feature

In [16]:
# 特徴量エンジニアリング関数
def add_feature(df, feature_func):
    return feature_func(df)

feature_functions = {
    'assistRate': lambda df: df.assign(assistRate=df['blueAssists'] / (df['blueKills'] + 1)),
    'blueHerald': lambda df: df.assign(blueHerald=df['blueEliteMonsters'] - df['blueDragons']).assign(blueHerald=lambda x: x['blueHerald'].clip(lower=0)),
    'eliteMonsterDragonRatio': lambda df: df.assign(eliteMonsterDragonRatio=df['blueEliteMonsters'] / (df['blueDragons'] + 1)),
    'logTotalGold': lambda df: df.assign(logTotalGold=np.log1p(df['blueTotalGold'])),
    'logTotalExperience': lambda df: df.assign(logTotalExperience=np.log1p(df['blueTotalExperience'])),
    'firstBloodKillRatio': lambda df: df.assign(firstBloodKillRatio=df['blueFirstBlood'] * df['blueKills']),
    'totalCombatPoints': lambda df: df.assign(totalCombatPoints=df['blueKills'] + df['blueDeaths'] + df['blueAssists']),
    'combatRatios': lambda df: df.assign(killRatio=df['blueKills'] / (df['blueKills'] + df['blueDeaths'] + df['blueAssists']),
                                           deathRatio=df['blueDeaths'] / (df['blueKills'] + df['blueDeaths'] + df['blueAssists']),
                                           assistRatio=df['blueAssists'] / (df['blueKills'] + df['blueDeaths'] + df['blueAssists'])),
    'teamStrength': lambda df: df.assign(teamStrength=df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']),
    'dragonKillImpact': lambda df: df.assign(dragonKillImpact=df['blueDragons'] / (df['blueKills'] + 1)),
    'eliteMonsterUtilization': lambda df: df.assign(eliteMonsterUtilization=df['blueEliteMonsters'] / (df['blueKills'] + df['blueAssists'] + 1)),
    'goldExperienceRatio': lambda df: df.assign(goldExperienceRatio=df['blueTotalGold'] / (df['blueTotalExperience'] + 1)),
    'teamEfficiency': lambda df: df.assign(teamEfficiency=(df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']) / (df['blueDeaths'] + 1)),
    'killToMonsterRatio': lambda df: df.assign(killToMonsterRatio=df['blueKills'] / (df['blueEliteMonsters'] + 1)),
    'avgGoldPerKill': lambda df: df.assign(avgGoldPerKill=df['blueTotalGold'] / (df['blueKills'] + 1)),
    'expToKillRatio': lambda df: df.assign(expToKillRatio=df['blueTotalExperience'] / (df['blueKills'] + 1)),
    'killsMinusDeaths': lambda df: df.assign(killsMinusDeaths=df['blueKills'] - df['blueDeaths']),
    'killDeathRatio': lambda df: df.assign(killDeathRatio=df['blueKills'] / (df['blueKills'] + df['blueDeaths'])),
    'avgExperiencePerKill': lambda df: df.assign(avgExperiencePerKill=df['blueTotalExperience'] / df['blueKills']),
}

- study_accuracy

In [17]:
# 特徴量の影響を評価する関数
def evaluate_feature_impact(feature_func):
    # 特徴量を追加
    X_train_modified = add_feature(X_train.copy(), feature_func)
    X_val_modified = add_feature(X_val.copy(), feature_func)

    # モデルの訓練
    lgb_train = lgb.Dataset(X_train_modified, y_train)
    lgb_val = lgb.Dataset(X_val_modified, y_val)
    model = lgb.train(
        best_params,
        lgb_train,
        valid_sets=lgb_val,
        num_boost_round=100,
        callbacks=[lgb.early_stopping(stopping_rounds=10)]
    )

    # 検証データに対する予測と精度の計算
    y_pred = model.predict(X_val_modified, num_iteration=model.best_iteration)
    y_pred = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

- 結果

In [18]:
# 各特徴量の精度を計算し、結果を出力
accuracy_results = {}
for feature_name, feature_func in feature_functions.items():
    accuracy = evaluate_feature_impact(feature_func)
    accuracy_results[feature_name] = accuracy
    print(f'{feature_name}_accuracy: {accuracy}')

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.205
assistRate_accuracy: 0.795
Training until validation scores don't improve for 10 rounds


Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.2025
blueHerald_accuracy: 0.7975
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[39]	valid_0's binary_error: 0.191875
eliteMonsterDragonRatio_accuracy: 0.808125
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.2025
logTotalGold_accuracy: 0.7975
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.2025
logTotalExperience_accuracy: 0.7975
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[36]	valid_0's binary_error: 0.19
firstBloodKillRatio_accuracy: 0.81
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[20]	valid_0's binary_error: 0.2075
totalCombatPoints_accuracy: 0.7925
Training until validation scores don't improve for 10 rounds
E

- 結果一覧

In [19]:
# 精度が高い順に辞書を並べ替え
sorted_accuracy_results = sorted(accuracy_results.items(), key=lambda x: x[1], reverse=True)

# ソートされた結果を表示
for feature_name, accuracy in sorted_accuracy_results:
    print(f'{feature_name}: {accuracy}')

teamStrength: 0.81125
avgGoldPerKill: 0.810625
firstBloodKillRatio: 0.81
killToMonsterRatio: 0.81
eliteMonsterDragonRatio: 0.808125
dragonKillImpact: 0.808125
eliteMonsterUtilization: 0.806875
goldExperienceRatio: 0.805
avgExperiencePerKill: 0.804375
killsMinusDeaths: 0.80375
teamEfficiency: 0.8025
combatRatios: 0.799375
blueHerald: 0.7975
logTotalGold: 0.7975
logTotalExperience: 0.7975
expToKillRatio: 0.796875
assistRate: 0.795
totalCombatPoints: 0.7925
killDeathRatio: 0.79


- 関数作成

In [20]:
def add_features(df):
    """
    与えられたデータフレームに対して特徴量エンジニアリングを行う関数。

    Args:
    df (DataFrame): 特徴量エンジニアリングを行うデータフレーム

    Returns:
    DataFrame: 新しい特徴量が追加されたデータフレーム
    """

    return df